[ข้อมูล TCAS](https://raw.githubusercontent.com/PyThaiNLP/thai-sentiment-analysis-dataset/master/tcas61.csv)

credit : [@nagarindkx](https://github.com/nagarindkx) ดู[โค้ดเดิม](https://github.com/nagarindkx/python/blob/master/python%2006%20Sentiment%20Analysis.ipynb) และอ่าน[บทความ](https://sysadmin.psu.ac.th/2019/01/15/python-06-sentiment-analysis-with-keras-tensorflow/)

In [1]:
pip install deepcut==0.6.1    

In [0]:
%tensorflow_version 1.x

In [3]:
import deepcut   # 0.6.1 for TensorFlow 1.x
deepcut.tokenize('ไปหามเหสี')

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








['ไป', 'หามเหสี']

In [0]:
%tensorflow_version 1.x
from keras.models import *
from keras.layers import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving a.csv to a (8).csv
User uploaded file "a.csv" with length 34233 bytes


In [6]:
df = pd.read_csv('a (3).csv')
df.head()

,text,label
0,มึงกล้าพูดรึป่าวว่าระบบการศึกษามันดีอ่ะ ถุ้ยเฟ...,neg
1,เบื่อเวลามาโพสตไรแบบนี้ชอบเป็นพวกที่ใช่โปรไฟล์...,neg
2,พ่อมึงเป็นติ่งรัฐบาลหรอสัส ที่เรียกเก็บตังแพงม...,neg
3,ใครก็ช่วยลบไอ้นี้ออกจากกลุ่มหน่อยครับ มันมาโพส...,neg
4,เครียดมากอะตอนนี้,neg


In [7]:
t = df['text'].apply(lambda s : deepcut.tokenize(s))
t.iloc[0]

['มึง',
 'กล้า',
 'พูด',
 'รึ',
 'ป่าว',
 'ว่า',
 'ระบบ',
 'การ',
 'ศึกษา',
 'มัน',
 'ดี',
 'อ่ะ',
 ' ',
 'ถุ้ยเฟส',
 'ยัง',
 'ปลอม',
 'เลย',
 'สัส']

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(t)
len(tokenizer.word_index)

802

In [9]:
tts = tokenizer.texts_to_sequences(t)
tts[0]

[95, 188, 124, 189, 350, 23, 57, 53, 125, 34, 58, 44, 1, 351, 35, 352, 20, 190]

In [10]:
maxlen = max([len(s) for s in tts])
print(maxlen)
x = pad_sequences(tts, maxlen=maxlen, padding="post")
x

168


array([[ 95, 188, 124, ...,   0,   0,   0],
       [154,  59,  15, ...,   0,   0,   0],
       [193,  95,  24, ...,   0,   0,   0],
       ...,
       [228,  91,   1, ...,   0,   0,   0],
       [228,  91, 140, ...,   0,   0,   0],
       [151,  99,   9, ...,   0,   0,   0]], dtype=int32)

In [11]:
maxlen = 50
x = x[:, :maxlen]
x.shape

(123, 50)

In [12]:
y = to_categorical([s for s in df['label'].apply(lambda s : 0 if s == 'neg' else 1)])
y[:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Train

In [13]:
vocab_size = len(tokenizer.word_index) + 1
print('vocab size', vocab_size)

np.random.seed(8)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=maxlen))
model.add(GRU(4))
model.add(Dense(y.shape[1], activation='relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

h = model.fit(x, y, epochs=300, verbose=1)
print(s, max(h.history['acc']), h.history['acc'][-1])

vocab size 803


Epoch 1/300
123/123 [==============================] - 1s 8ms/step - loss: 6.8898 - acc: 0.5000
Epoch 2/300
123/123 [==============================] - 0s 927us/step - loss: 4.2133 - acc: 0.5000
Epoch 3/300
123/123 [==============================] - 0s 974us/step - loss: 3.8569 - acc: 0.5000
Epoch 4/300
123/123 [==============================] - 0s 989us/step - loss: 3.6894 - acc: 0.5000
Epoch 5/300
123/123 [==============================] - 0s 1ms/step - loss: 2.2933 - acc: 0.5000
Epoch 6/300
123/123 [==============================] - 0s 1ms/step - loss: 1.5754 - acc: 0.5000
Epoch 7/300
123/123 [==============================] - 0s 1ms/step - loss: 1.4350 - acc: 0.5000
Epoch 8/300
123/123 [==============================] - 0s 1ms/step - loss: 1.3537 - acc: 0.5000
Epoch 9/300
123/123 [==============================] - 0s 1ms/step - loss: 1.2926 - acc: 0.5000
Epoch 10/300
123/123 [==============================] - 0s 970us/step - loss: 1.2481 - acc: 0.5000
Epoch 11/300
1

NameError: ignored

In [14]:
test=[
    'ไม่ติดอ่ะ',
    'จน เครียด กินนม',
    'นี่แหละคือความกดดัน',
    'ทำผมเสียเวลามาก',
    'ติดแล้ว',
    'ทุกคนสู้ต่อไป',
    'ดีใจจัง สอบติดแล้ว',
    'จะได้เป็นนิสิตเกษตร',
]
x_data_test = np.array(test)

x_test=[]
for t in x_data_test:                   
    s = []                          # t :                        'I do not like it'
    for w in deepcut.tokenize(t):   # text_to_word_sequence(t) : ['I', 'do', 'not', 'like', 'it']
        s.append(tokenizer.word_index[w] if w in tokenizer.word_index else 0)
    x_test.append(s)
x_test = pad_sequences(x_test,maxlen=50, padding='post')

model.predict_classes(x_test)

array([0, 1, 0, 0, 1, 1, 1, 1])

In [0]:
vocab_size = len(tokenizer.word_index) + 1

for s in range(7,100):
  np.random.seed(s)

  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=maxlen))
  model.add(GRU(4))
  model.add(Dense(y.shape[1], activation='relu'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  h = model.fit(x, y, epochs=400, verbose=0)
  print(s, max(h.history['acc']), h.history['acc'][-1])
    